In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [4]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1721406456090, experiment_id='1', last_update_time=1721406456090, lifecycle_stage='active', name='nyc-experiment', tags={}>

In [5]:
#Training data 
dft=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
dfv=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

the january data has 19


In [7]:
#calculate duration using guide notebook
dft['duration'] = dft.tpep_dropoff_datetime - dft.tpep_pickup_datetime
dft.duration = dft.duration.apply(lambda td: td.total_seconds() / 60)
dfv['duration'] = dfv.tpep_dropoff_datetime - dfv.tpep_pickup_datetime
dfv.duration = dfv.duration.apply(lambda td: td.total_seconds() / 60)

In [8]:
#we drop outliers using the same principle from the notebook
dftf = dft[(dft.duration >= 1) & (dft.duration <= 60)]
dfv = dfv[(dfv.duration >= 1) & (dfv.duration <= 60)]


In [9]:
dftf['PU_DO'] = dftf['PULocationID'].apply(str) + '_' + dftf['DOLocationID'].apply(str)
dfv['PU_DO'] = dfv['PULocationID'].apply(str) + '_' + dfv['DOLocationID'].apply(str)

/tmp/ipykernel_40939/1062032864.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftf['PU_DO'] = dftf['PULocationID'].apply(str) + '_' + dftf['DOLocationID'].apply(str)


In [10]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dftf[categorical] = dftf[categorical].astype(str)
#With the hot encoding we have 2xn matrix
dfv[categorical] = dfv[categorical].astype(str)

/tmp/ipykernel_40939/1811195592.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftf[categorical] = dftf[categorical].astype(str)


In [11]:
train_dicts = dftf[categorical + numerical].to_dict(orient='records')
val_dicts = dfv[categorical + numerical].to_dict(orient='records')

In [12]:

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)






#y_pred = lr.predict(X_train)

#root_mean_squared_error(y_train, y_pred)

In [13]:
target = 'duration'
y_train = dftf[target].values
y_val = dfv[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [15]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [16]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Jesus")
    
    mlflow.log_param("train-data-url","https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
    mlflow.log_param("validation-data-url","https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")
    
    alpha=0.01
    mlflow.log_param("alpha",alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse=root_mean_squared_error(y_val, y_pred)
    mlflow.log_param("RMSE",rmse)

In [18]:
print(rmse)
#so we choose 7.81 as our answer

9.395699095576239
